# Text summarization

## Task

- find extractive model
- choose relevant dataset and metrics
- final solution takes text and number of sentences and gives out text

## Analysis


- difference between abstractive and extractive models: in abstractive summarization, target summaries contains words or phrases that were not in the original text and usually require various text rewriting operations to generate, while extractive approaches form summaries by copying and concatenating the most important spans (usually sentences) in a document
- benchmark datasets:
  - for single-lingual:
    - CNN/DailyMail (contains news article and associated highlights, i.e., a few bullet points giving a brief overview of the article)
  - for multilingual:
    - [MLSUM](https://arxiv.org/pdf/2004.14900.pdf)
    - [XL-Sum](https://aclanthology.org/2021.findings-acl.413.pdf) (abstractive)
- metrics:
  - ROUGE-1 refers to the overlap of unigram (each word) between the system and reference summaries.
  - ROUGE-2 refers to the overlap of bigrams between the system and reference summaries.
  - ROUGE-L: Longest Common Subsequence (LCS) based statistics. Longest common subsequence problem takes into account sentence level structure similarity naturally and identifies longest co-occurring in sequence n-grams automatically.
- single-lingual models:
  - abstractive:
    - [BART](https://arxiv.org/pdf/1910.13461.pdf)
    - [BART+R3F](https://openreview.net/pdf?id=OQ08SN70M1V)
    - [BART+R-Drop](https://arxiv.org/pdf/2106.14448v1.pdf)
  - extractive:
    - [BertSumExt](https://github.com/nlpyang/PreSumm)
    - [MatchSum](https://github.com/maszhongming/MatchSum)

Abstractive models based on BART, extractive on BERT.
BART has architecture with a bidirectional encoder (like BERT) and a left-to-right decoder (like GPT).

| Model              | R1    | R2    | RL    |
|--------------------|-------|-------|-------|
| Random Transformer | 38.27 | 15.03 | 35.48 |
| BertSumExt         | 43.85 | 20.34 | 39.90 |
| BART               | 44.16 | 21.28 | 40.90 |
| BART+R3F           | 44.38 | 21.53 | 41.17 |
| MatchSum           | 44.41 | 20.86 | 40.55 |
| BART+R-Drop        | 44.51 | 21.58 | 41.24 |

- multilingual models:
  - abstractive:
     - Pointer-Generator (based on LSTM)
     - M-BERT
  - extractive:
    - Oracle (Extracts the sentences, within the input text, that maximise a given metric given the reference summary)
    - TextRank (Computes the co-similarities between all the sentences
in the input text)
    - Lead-3 (Simply selects the three first sentences
from the input text)



## Solution



In [ ]:
!pip install datasets
!pip install summa
!pip install rouge_score
!pip install tqdm

### Multilingual


Try to reproduce multilingual solution that will use MLSUM as dataset, TextRank as baseline model and model based on BERT as proposed model.

#### Dataset


In [2]:
from datasets import load_dataset, load_metric

In [ ]:
data_en_test = load_dataset("cnn_dailymail", '3.0.0', split='test[:100]')

In [ ]:
data_de_test = load_dataset("mlsum", "de", split='test[:100]')

#### Baseline

In [5]:
from summa.summarizer import summarize
import tqdm.notebook as tq

In [6]:
results = {}

In [7]:
rouge_de = load_metric('rouge')

for el in tq.tqdm(data_de_test):
  model_predictions = summarize(el['text'], language='german')
  rouge_de.add_batch(predictions=[model_predictions], references=[el['summary']])

results['baseline_de'] = rouge_de.compute()

  0%|          | 0/100 [00:00<?, ?it/s]

In [8]:
rouge_en = load_metric('rouge')

for el in tq.tqdm(data_en_test):
  model_predictions = summarize(el['article'], language='english')
  rouge_en.add_batch(predictions=[model_predictions], references=[el['highlights']])

results['baseline_en'] = rouge_en.compute()

  0%|          | 0/100 [00:00<?, ?it/s]

#### BERT

As we can see in analysis, the popular single-lingual models are based on BERT. So it will be a good solution to use multilingual BERT as basis.

Current solution utilizes the BERT model for text embeddings and K-Means clustering to identify sentences closest to the centroid for summary selection

In [ ]:
!pip install spacy==2.1.3
!pip install transformers==3.3.0
!pip install sentencepiece==0.1.95
!pip install neuralcoref==4.0

In [ ]:
!pip install bert-extractive-summarizer==0.4.2

In [11]:
from summarizer import Summarizer
from transformers import BertTokenizer, BertModel

In [12]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
model = BertModel.from_pretrained('bert-base-multilingual-uncased', output_hidden_states=True)

Downloading:   0%|          | 0.00/872k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/672M [00:00<?, ?B/s]

In [13]:
sum_model = Summarizer(custom_model=model, custom_tokenizer=tokenizer)

In [14]:
rouge_de = load_metric('rouge')

for el in tq.tqdm(data_de_test):
  model_predictions = sum_model(el['text'])
  rouge_de.add_batch(predictions=[model_predictions], references=[el['summary']])

results['bert_de'] = rouge_de.compute()

  0%|          | 0/100 [00:00<?, ?it/s]

In [15]:
rouge_en = load_metric('rouge')

for el in tq.tqdm(data_en_test):
  model_predictions = sum_model(el['article'])
  rouge_en.add_batch(predictions=[model_predictions], references=[el['highlights']])

results['bert_en'] = rouge_en.compute()

  0%|          | 0/100 [00:00<?, ?it/s]

### Visualization results

In [16]:
import pandas as pd

In [17]:
def show_results():
    sorted_res = dict(sorted(results.items(), key=lambda item: item[1]['rougeL'].mid.fmeasure))
    v = list(sorted_res.values())
    fst = list(map(lambda x: x['rougeL'].mid.fmeasure, v))
    df = pd.DataFrame({"name": list(sorted_res.keys()), "rougeL": fst})
    with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
        display(df)

In [18]:
show_results()

,name,rougeL
0,bert_en,0.129398
1,baseline_en,0.131914
2,baseline_de,0.136253
3,bert_de,0.172937


In general, BERT shows better results than baseline model and also BERT is more promising for future due to it's good results in single-lingual summarization.

### Testing



In [19]:
en_body = '''
The Chrysler Building, the famous art deco New York skyscraper, will be sold for a small fraction of its previous sales price.
The deal, first reported by The Real Deal, was for $150 million, according to a source familiar with the deal.
Mubadala, an Abu Dhabi investment fund, purchased 90% of the building for $800 million in 2008.
Real estate firm Tishman Speyer had owned the other 10%.
The buyer is RFR Holding, a New York real estate company.
Officials with Tishman and RFR did not immediately respond to a request for comments.
It's unclear when the deal will close.
The building sold fairly quickly after being publicly placed on the market only two months ago.
The sale was handled by CBRE Group.
The incentive to sell the building at such a huge loss was due to the soaring rent the owners pay to Cooper Union, a New York college, for the land under the building.
The rent is rising from $7.75 million last year to $32.5 million this year to $41 million in 2028.
Meantime, rents in the building itself are not rising nearly that fast.
While the building is an iconic landmark in the New York skyline, it is competing against newer office towers with large floor-to-ceiling windows and all the modern amenities.
Still the building is among the best known in the city, even to people who have never been to New York.
It is famous for its triangle-shaped, vaulted windows worked into the stylized crown, along with its distinctive eagle gargoyles near the top.
It has been featured prominently in many films, including Men in Black 3, Spider-Man, Armageddon, Two Weeks Notice and Independence Day.
The previous sale took place just before the 2008 financial meltdown led to a plunge in real estate prices.
Still there have been a number of high profile skyscrapers purchased for top dollar in recent years, including the Waldorf Astoria hotel, which Chinese firm Anbang Insurance purchased in 2016 for nearly $2 billion, and the Willis Tower in Chicago, which was formerly known as Sears Tower, once the world's tallest.
Blackstone Group (BX) bought it for $1.3 billion 2015.
The Chrysler Building was the headquarters of the American automaker until 1953, but it was named for and owned by Chrysler chief Walter Chrysler, not the company itself.
Walter Chrysler had set out to build the tallest building in the world, a competition at that time with another Manhattan skyscraper under construction at 40 Wall Street at the south end of Manhattan. He kept secret the plans for the spire that would grace the top of the building, building it inside the structure and out of view of the public until 40 Wall Street was complete.
Once the competitor could rise no higher, the spire of the Chrysler building was raised into view, giving it the title.
'''

In [20]:
sum_model(en_body)

"The Chrysler Building, the famous art deco New York skyscraper, will be sold for a small fraction of its previous sales price. The rent is rising from $7.75 million last year to $32.5 million this year to $41 million in 2028. Still there have been a number of high profile skyscrapers purchased for top dollar in recent years, including the Waldorf Astoria hotel, which Chinese firm Anbang Insurance purchased in 2016 for nearly $2 billion, and the Willis Tower in Chicago, which was formerly known as Sears Tower, once the world's tallest. Blackstone Group (BX) bought it for $1.3 billion 2015. He kept secret the plans for the spire that would grace the top of the building, building it inside the structure and out of view of the public until 40 Wall Street was complete."

### TOTAL

| Pros                                                                   | Cons         |
|------------------------------------------------------------------------|--------------|
| Multilingual solution                                                  | Bad accuracy |
| Based on BERT, which has good results  in single-lingual summarization |              |
| Better results than baseline                                           |              |

### ToDo

- explore and try other clustering methods
- make comparison with abstractive models
- use mT5 and other relevant models for text embeddings